In [4]:
import os

In [5]:
%pwd

'c:\\Projects\\End-to-End_Pricing\\reseacrh'

In [6]:
os.chdir("../")

In [7]:
%pwd

'c:\\Projects\\End-to-End_Pricing'

In [8]:
from pathlib import Path
from dataclasses import dataclass
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: Path
    local_data_file: Path
    unzip_dir: Path

In [9]:
from src.house_reg._utils.common import read_yaml,create_directories

[2024-12-26 19:56:25,865: INFO: __init__: Logging setup successful.]


In [10]:
from src.house_reg.constant import *

In [18]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_PATH, params_file_path=PARAMS_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        dataingestionconfig = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_url=str(config.source_url),
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return dataingestionconfig
    
    


#### component

In [19]:
import gdown
import zipfile
from src.house_reg import logger

In [24]:
class DataIngestion:

    def __init__(self,config:DataIngestionConfig):
        self.config= config

    def download_file(self)->str:
        '''fetch Data from url
        '''
        try:
            dataset_url=self.config.source_url
            zip_download_dir= self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading the dataset:{dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
            gdown.download(download_url, str(zip_download_dir), quiet=False)

            logger.info(f"Downloaded Data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path : str
        Extracts the zip file into the data directory

        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [25]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-26 20:05:40,778: INFO: common: yaml file:config\config.yaml loaded successfully]
[2024-12-26 20:05:40,780: INFO: common: yaml file:params.yaml loaded successfully]
[2024-12-26 20:05:40,781: INFO: common: created directory:artifacts]
[2024-12-26 20:05:40,782: INFO: common: created directory:artifacts/data_ingestion]
[2024-12-26 20:05:40,783: INFO: 2874372300: Downloading the dataset:https://drive.google.com/file/d/1hXDTuXwUl0l2E3GM-ZHRwMHELhZnaxrW/view?usp=sharing into file artifacts\data_ingestion\data.zip]


Downloading...
From: https://drive.google.com/uc?export=download&id=1hXDTuXwUl0l2E3GM-ZHRwMHELhZnaxrW
To: c:\Projects\End-to-End_Pricing\artifacts\data_ingestion\data.zip

100%|██████████| 4.74k/4.74k [00:00<?, ?B/s]

[2024-12-26 20:05:45,147: INFO: 2874372300: Downloaded Data from https://drive.google.com/file/d/1hXDTuXwUl0l2E3GM-ZHRwMHELhZnaxrW/view?usp=sharing into file artifacts\data_ingestion\data.zip]
